# Installing dependencies


In [ ]:
!pip install -q accelerate -U
!pip install -q bitsandbytes -U
!pip install -q trl -U
!pip install -q peft -U
!pip install -q transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatib

# test some base model


## TinyLlama-1.1B-Chat-v1.0

In [ ]:
from transformers import pipeline
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
pipe = pipeline(task='text-generation', model=model_name, device='cuda')


# prompt
# <|user|>, <|assistant|>

prompt = """<|user|>
Diego baked 12 cakes for his sister's birthday. Donald also baked 4 cakes, but ate 1 while waiting for the party to start. How many cakes are left?.</s>
<|assistant|>
"""





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
prompt = """<|user|>
Diego baked 12 cakes for his sister's birthday. Donald also baked 4 cakes, but ate 1 while waiting for the party to start. How many cakes are left?.</s>
<|assistant|>
"""
output = pipe(prompt)
output

[{'generated_text': "<|user|>\nDiego baked 12 cakes for his sister's birthday. Donald also baked 4 cakes, but ate 1 while waiting for the party to start. How many cakes are left?.</s>\n<|assistant|>\nDiego and Donald each baked 12 cakes for their sister's birthday, so there are 12 cakes left."}]

# Load the datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cognitivecomputations/dolphin-coder",trust_remote_code=True, split="train")
dataset = dataset.shuffle(seed=0).select(range(30_000))

Generating train split:   0%|          | 0/109118 [00:00<?, ? examples/s]

In [ ]:
dataset_meta_math = load_dataset("meta-math/MetaMathQA",split="train")
dataset_meta_math = dataset_meta_math.shuffle(seed=0).select(range(30_000))
dataset_magic_coder = load_dataset("ise-uiuc/Magicoder-Evol-Instruct-110K",split="train")


Generating train split:   0%|          | 0/395000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/111183 [00:00<?, ? examples/s]

In [ ]:
dataset_magic_coder = dataset_magic_coder.shuffle(seed=0).select(range(50_000))

In [ ]:
from transformers import AutoTokenizer

# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# template_tokenizer = AutoTokenizer.from_pretrained(model_name)

# Dictionary yang mengatur key untuk dataset yang berbeda
arg = {
    "dolphine": ['question', 'response'],
    "magic-coder": ['instruction', 'response'],
    "meta-math": ['original_question','response']
}

def format_prompt_(example, dataset_type):
    """Format the prompt using the user and assistant format based on dataset type."""


    user_input = example[arg[dataset_type][0]]
    assistant_output = example[arg[dataset_type][1]]
    # Format ke dalam format UltraChat
    prompt = f"<|user|> {user_input} <|assistant|> {assistant_output}"

    return {'text': prompt}

# load all dataset





In [ ]:


dataset_meta_math = dataset_meta_math.map(lambda example: format_prompt_(example, 'meta-math'))

dataset_dolphine = dataset.map(lambda example: format_prompt_(example, 'dolphine'))

dataset_magic_coder = dataset_magic_coder.map(lambda example: format_prompt_(example, 'magic-coder'))

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset_magic_coder[0]['text']

'<|user|> I\'ve got a menu that contains, among other things, some most-recently-used file paths.  The paths to these files can be long, so the text sometimes gets clipped like "C:\\Progra...\\foo.txt"\n\nI\'d like to pop a tooltip with the full path when the user hovers over the item, but this doesn\'t seem possible with the [Tooltip](http://msdn.microsoft.com/en-us/library/system.windows.forms.tooltip.aspx) class in .NET 2.0.\n\nAm I missing something obvious? <|assistant|> You are not missing something obvious. ToolTips in the .NET 2.0 framework are not natively supported for menu items.\n\nHowever, you might be able to achieve this feature by creating an owner-draw menu and handling the `MouseHover` event yourself to display a custom tooltip. Be aware that creating an owner-draw menu requires advanced programming.\n\nAnother approach varies a bit since it doesn\'t use tooltips, but achieves the same functionality you want, which is showing complete file paths:\n\nAdd the file path 

In [ ]:
dataset_magic_coder

Dataset({
    features: ['instruction', 'response', 'text'],
    num_rows: 50000
})

# model configuration

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig



In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_size="left"

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<PAD>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    quantization_config=bnb_config
)

In [ ]:
model.config.use_cache=False
model.config.pretraining_tp=1

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05

# Prepare LoRA Configuration for PEFT Fine tuning


In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)

In [ ]:
w = 2048*256
a = 2048*16
b = 16*256

w, a, b, a+b, (a+b)/w

(524288, 32768, 4096, 36864, 0.0703125)

# Model fine tuning


In [ ]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer

output_dir = "train_dir"

args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset = dataset_magic_coder,
    dataset_text_field='text',
    tokenizer = tokenizer,
    args=args,
    max_seq_length=512,
    peft_config = peft_config
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Step,Training Loss
10,1.408200
20,1.371800
30,1.323900
40,1.252200
50,1.280100
60,1.265400
70,1.451500
80,1.272100
90,1.255400
100,1.322900


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

In [ ]:
trainer.model.save_pretrained("TinyLlama-1.1B-ChatV1.0-qlora")

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-ChatV1.0-qlora",
    device_map='auto'
)

merged_model = model.merge_and_unload()

In [ ]:
from transformers import pipeline

prompt = """<|user|>
code me program to find prime number.</s>
<|assistant|>
"""

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_size="left"

pipe = pipeline(task='text-generation', model=merged_model, tokenizer=tokenizer)
output = pipe(prompt)
print(output[0]['generated_text'])

<|user|>
Diego baked 12 cakes for his sister's birthday. Donald also baked 4 cakes, but ate 1 while waiting for the party to start. How many cakes are left.</s>
<|assistant|>
Diego baked 12 cakes.
Donald ate 1 cake.
So, there are 12 - 1 = 11 cakes left.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The answer is: 11

The value of x is 11.
#### 11
The